# Test Modules

In [ ]:
import os
import sys
import subprocess
import time

# Change to project directory
os.chdir('/scratch/edk202/word2gm-fast/notebooks')
os.chdir("..")

# Clean TensorFlow import with complete silencing (non-deterministic by default)
from src.word2gm_fast.utils import (
    import_tensorflow_silently, 
    log_tf_to_file, 
    run_silent_subprocess
)

tf = import_tensorflow_silently(deterministic=False)
print(f"✅ TensorFlow {tf.__version__} imported silently (non-deterministic mode)")

# Setup notebook environment for testing
from word2gm_fast.utils.notebook_setup import setup_data_preprocessing_notebook, enable_autoreload

# Set up environment with CPU-only mode (optimal for testing and data processing)
env = setup_data_preprocessing_notebook()

# Enable auto-reload for development
enable_autoreload()

# Extract useful utilities
tf = env['tf']
print_resource_summary = env['print_resource_summary']

print(f"TensorFlow {tf.__version__} imported successfully")
print("Test environment ready!")

# Import additional testing modules not included in standard setup
import subprocess
import time

# Import data pipeline modules for testing
from word2gm_fast.dataprep.corpus_to_dataset import make_dataset
from word2gm_fast.dataprep.index_vocab import make_vocab
from word2gm_fast.dataprep.dataset_to_triplets import build_skipgram_triplets
from word2gm_fast.dataprep.tfrecord_io import (
    save_pipeline_artifacts,
    load_pipeline_artifacts,
    write_triplets_to_tfrecord,
    load_triplets_from_tfrecord,
    write_vocab_to_tfrecord,
    load_vocab_from_tfrecord
)

print("All testing modules imported successfully")

✅ TensorFlow 2.19.0 imported silently (non-deterministic mode)


In [ ]:
# Clean test execution - no duplicates
print("🧪 Testing all modules...")

modules_to_test = [
    ('tests.test_corpus_to_dataset', 'Corpus processing'),
    ('tests.test_index_vocab', 'Vocabulary indexing'), 
    ('tests.test_dataset_to_triplets', 'Triplet generation'),
    ('tests.test_tfrecord_io', 'TFRecord I/O')
]

test_results = []
for test_module, test_name in modules_to_test:
    if 'triplets' in test_module:
        cmd_result = run_silent_subprocess(
            ['python', '-m', 'unittest', '-b', test_module], 
            deterministic=False, capture_output=True, text=True
        )
    else:
        cmd_result = run_silent_subprocess(
            ['python', '-m', 'unittest', '-b', test_module], 
            capture_output=True, text=True
        )
    
    status = "PASS" if cmd_result.returncode == 0 else "FAIL"
    print(f"  {test_name}: {status}")
    test_results.append(cmd_result.returncode == 0)

passes = sum(test_results)
total = len(test_results)
print(f"\nResult: {passes}/{total} tests passed")
if passes == total:
    print("✅ All tests successful!")
else:
    print("❌ Some tests failed")